In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


# First lets look at the GOS

In [51]:
# %%timeit -r 1
nviews = 493 #493 #247 #165 #124 for the others

s = fc.Spectrum()
s.load('Varian_truebeam')
s.x[0] = 1; s.x[1] = 2
s.attenuate(0.,fc.get_mu(z=13))

kernel = fc.Kernel(s,'CuGOS-784-micrometer')
kernel.add_focal_spot(1.2)
angles = np.linspace(0,np.pi*2,nviews)

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,mgy=54*1.3907/(507),bowtie=True,filter='FF0') # should be 507
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[5].shape,r=5.7,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[5],im)

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 162.95898755328747 number of photons 0.02636808212587352


## Now lets look at the bcca data

In [52]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_6x_300_rared.npy')
recon_slice = np.rot90(np.mean(bcca_recon.T[35:36],0),2)
im = create_mask(recon_slice.shape,off =[-0.15,0], r= 5.75,radius=0.3)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in [-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04]:
    reses.append(minimize(HU_fun,[contrast_fc[0],contrast_fc[5]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]

x = [.853,0.945,.998,1.147,1.36,1.87]
contrast_fc_scaled = (contrast_fc - res.x[0])*(res.x[1])
image_fc_scaled = np.roll((phantom.img[4]- res.x[0])*(res.x[1]),5)

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.imshow(recon_slice,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(222)
plt.imshow(image_fc_scaled,cmap='gray',vmin=0.001,vmax=0.007)
plt.axis('off')
plt.subplot(223)
plt.plot(x,contrast_fc_scaled,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,contrast_bcca,'s',c='darkorange',label='Star-lack')
plt.subplot(224)
plt.plot(recon_slice[256,7:-7])
plt.plot(np.rot90(image_fc_scaled,0)[257,7:-7])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
np.save('fc_images/fc_image_gos_6x_124',image_fc_scaled)

In [76]:
plt.imsave('im_for_imagej.png',np.hstack((recon_slice,image_fc_scaled)))
!imagej im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]



# Now lets look at the kV data 

In [53]:
# %%timeit -r 1
nviews = 222 #887 # 222 296 444 887 That is how many are in the file 
s = fc.calculate_spectrum(100, 14, 3, 100,monitor=None)
s.attenuate(5.6,fc.get_mu(z=13)) #0.4
s.attenuate(0.089,fc.get_mu(z=22))
kernel = fc.Kernel(s,'CsI-784-micrometer')
# angles = np.linspace(0.1,np.pi*2,nviews)        

(19, 19, 19) 19


In [61]:
angles = np.load('/home/xcite/MV_CBCT/angles_kv.npy')[::4]

In [62]:
conversion = 1.07765#*(0.511/0.784)**2

fudge = 1 # 0.018
# !!!!! Scatter is off
phantom = fc.Catphan_404()#pitch = 0.511)
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon_revised','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,scat_on=False,mgy=fudge*21.1/(nviews),bowtie=True,filter='bowtie_mod') #bowtie_mod 1.077 conversion
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[4].shape,r=5.75,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[4],im)

bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 1734.0094827566788 number of photons 0.19327890235626993


In [63]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kV_fourth.npy').T#recon_kv_jan_2021.npy')
recon_slice = np.rot90(np.mean(bcca_recon.T[39:47],0)).copy()
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in np.arange(-0.2,0.2,0.01):
    reses.append(minimize(HU_fun,[contrast_fc[2],contrast_fc[-3]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
    
fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (phantom.img[4] - res.x[0])*(res.x[1])

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
plt.plot(recon_slice[256,:],c='darkorange')
plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
print(phantom.geomet)

TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1500 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 64 512]
Size of each pixel (dDetector) = [0.784 0.784] mm
Total size of the detector (sDetector) = [ 50.176 401.408] mm
-----
Image parameters
Number of voxels (nVoxel) = [ 10 512 512]
Total size of the image (sVoxel) = [200 200 200] mm
Size of each voxel (dVoxel) = [20.        0.390625  0.390625] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad


In [31]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_kV_fourth.npy').T#recon_kv_jan_2021.npy')
recon_slice = np.rot90(bcca_recon.T[-19]).copy()
im = create_mask(recon_slice.shape,r=5.8,off = [0.2,0.05],rot = 1)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in [-0.64,-0.23,-0.08,-0.01,0,0.01,0.06,0.23,0.64]:
    reses.append(minimize(HU_fun,[contrast_fc[2],contrast_fc[-3]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]
    
fc_data = (contrast_fc - res.x[0])*(res.x[1])
fc_image = (phantom.img[4] - res.x[0])*(res.x[1])

x = [.853,0.945,.998,1.147,1.36,1.87]

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.plot(x,contrast_bcca,'s',c='darkorange',label='')
plt.plot(x,fc_data,'*',c='cornflowerblue',label='fastCAT')
plt.legend()
plt.subplot(222)
plt.plot(recon_slice[256,:],c='darkorange')
plt.plot(np.rot90(fc_image,0)[256,:],c='cornflowerblue')
plt.subplot(223)
plt.imshow(np.rot90(fc_image,0),cmap='gray',vmin=0.013,vmax=0.027)
plt.subplot(224)
plt.imshow(np.rot90(recon_slice,2),cmap='gray',vmin=0.013,vmax=0.027)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
plt.imsave('data/im_for_imagej.png',np.hstack((recon_slice,fc_image)))
!imagej data/im_for_imagej.png

Open other images in this ImageJ panel as follows:
  imagej -p 1 <image1> [<image2> ... <imageN>]



# Now let's take a look at the images in terms of CNR

In [58]:
from glob import glob

In [90]:
im = create_mask([512,512],r=5.7,radius=0.4)
im2 = create_mask([512,512],off =[-0.15,0], r= 5.75,radius=0.3)

CNRs = []

plt.figure()

exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_6x_300_*.npy'))[np.array([0,1,3,2])]

ii = 0
for file_fc, file_exp in zip(sorted(glob('fc_images/*.npy'))[-1::-1],exp_files):
    
    im_fc = np.load(file_fc)
    im_exp = np.rot90(np.load(file_exp).T[35],2)
                             
    contrast_fc, CNR_fc, noise_fc = return_CNR(np.roll(im_fc,-5),im)
    contrast_bcca, CNR_exp, noise_bcca = return_CNR(im_exp,im2)#,show_map=True)
                             
    CNRs.append(CNR_fc)
    plt.plot(ii,np.mean(CNR_fc),'b*',label=file_fc.split('/')[-1][-7:-4] + ' fc')
    plt.plot(ii+0.1,2.6*np.mean(CNR_exp),'r*',label=file_fc.split('/')[-1][-7:-4] +' exp')
    
    ii += 1

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets look at the 2.5 MV

In [2]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [40]:
# %%timeit -r 1
nviews =  203 #493 #247 #165 #124 for the others

s = fc.Spectrum()
s.load('W_spectrum_25')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
# kernel.add_focal_spot(1.2)
angles = np.load('/home/xcite/MV_CBCT/angles_25x.npy')

phantom = fc.Catphan_404()
phantom.phan_map = ['air','G4_NYLON-8062','CATPHAN_Acrylic','CATPHAN_Acrylic',
                    'CATPHAN_Delrin_revised','CATPHAN_Acrylic','CATPHAN_Teflon_revised',
                    'air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic','air','CATPHAN_Teflon','air','air','air','air'] 
phantom.return_projs(kernel,s,angles,mgy=600*3/nviews,bowtie=True,filter='FF0') # should be 507
phantom.reconstruct('FDK',filt='ram_lak')

im = create_mask(phantom.img[5].shape,r=5.7,radius=0.4)
contrast_fc, CNR_fc, noise_fc = return_CNR(phantom.img[5],im,show_map=True)

(19, 41, 41) 19
bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 17874.726480268397 number of photons 0.01015508406165716


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Now lets look at the bcca data

In [41]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_25x_60.npy')
recon_slice = np.rot90(bcca_recon.T[20],2)
im = create_mask(recon_slice.shape,off =[0.15,0], r= 5.75,radius=0.3)
contrast_bcca, CNR_bcca, noise_bcca = return_CNR(recon_slice,im)#,show_map=True)

In [42]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

reses = []
rem = []

for off in [-0.04,-0.03,-0.02,-0.01,0,0.01,0.02,0.03,0.04]:
    reses.append(minimize(HU_fun,[contrast_fc[0],contrast_fc[5]+off],args=(contrast_fc,contrast_bcca)))
    rem.append(reses[-1].fun)
    
res = reses[np.argmin(rem)]

x = [.853,0.945,.998,1.147,1.36,1.87]
contrast_fc_scaled = (contrast_fc - res.x[0])*(res.x[1])
image_fc_scaled = np.roll((phantom.img[4]- res.x[0])*(res.x[1]),5)

plt.figure(figsize=[8,8])
plt.subplot(221)
plt.imshow(recon_slice,cmap='gray',vmin=0.003,vmax=0.014)
plt.axis('off')
plt.subplot(222)
plt.imshow(image_fc_scaled,cmap='gray',vmin=0.003,vmax=0.014)
plt.axis('off')
plt.subplot(223)
plt.plot(x,contrast_fc_scaled,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,contrast_bcca,'s',c='darkorange',label='Star-lack')
plt.subplot(224)
plt.plot(recon_slice[256,7:-7])
plt.plot(np.rot90(image_fc_scaled,0)[257,7:-7])
plt.tight_layout()

/home/xcite/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now let's take a look at the images in terms of CNR

In [2]:
from glob import glob

In [90]:
im = create_mask([512,512],r=5.7,radius=0.4)
im2 = create_mask([512,512],off =[-0.15,0], r= 5.75,radius=0.3)

CNRs = []

plt.figure()

exp_files = np.array(glob('/home/xcite/MV_CBCT/recon_kV*.npy'))[np.array([0,1,3,2])]

ii = 0
for file_fc, file_exp in zip(sorted(glob('fc_images/*.npy'))[-1::-1],exp_files):
    
    im_fc = np.load(file_fc)
    im_exp = np.rot90(np.load(file_exp).T[35],2)
                             
    contrast_fc, CNR_fc, noise_fc = return_CNR(np.roll(im_fc,-5),im)
    contrast_bcca, CNR_exp, noise_bcca = return_CNR(im_exp,im2)#,show_map=True)
                             
    CNRs.append(CNR_fc)
    plt.plot(ii,np.mean(CNR_fc),'b*',label=file_fc.split('/')[-1][-7:-4] + ' fc')
    plt.plot(ii+0.1,2.6*np.mean(CNR_exp),'r*',label=file_fc.split('/')[-1][-7:-4] +' exp')
    
    ii += 1

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:

no = 0.784
npt = 512

np2 = 512
no2 = 0.511

pts2 = np.linspace(0,np2*no2,np2) - np2/2*no2
pts = np.linspace(0,npt*no,npt) - npt/2*no

p2 = np.zeros_like(phantom.fs)
for ii in range(phantom.fs.shape[0]):
    p2[ii] = np.interp(pts2,pts,phantom.fs[ii])

plt.figure()
plt.plot(pts,phantom.fs.T)
plt.plot(pts2,p2.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [39]:
len(p2.shape)

2

In [40]:
kernel.pitch

0.784